In [1]:
import pandas as pd

importons le jeux de donneée de l'annee juste avant le LMD


In [2]:
data= pd.read_csv('C:/Users/HP/Desktop/mon_stage_senegal/data/Etudiants_2001_2024/Liste_globale_des_etudiants_tous_les_champs20092010.csv' , encoding='latin1' ,sep=';')


affichons les colonnes pour comprendre la structure du fichier

In [3]:
data.columns

Index(['NUMERO', 'NUMERO_TABLE', 'INE', 'NUMERO_IDENTITIFICATION', 'NOM',
       'PRENOM', 'DATE_DE_NAISSANCE', 'LIEU_DE_NAISSANCE',
       'MAIL_INSTITUTIONNEL', 'SEXE', 'MENTION_BACC', 'ANNEE_BACC',
       'PAYS_DE_NAISSANCE', 'NATIONALITE', 'SERIE_BACC', 'REGION_DE_NAISSANCE',
       'ETABLISSMENT_CODE', 'ETABLISSEMENT', 'NIVEAU_SECTION',
       'SIGLE_NIVEAU_SECTION', 'COHORTE', 'ETAT_INSCRIPTION',
       'ANNEE_INSCRIPTION', 'ANNEE_UNIVERSITAIRE', 'TYPE_FORMATION',
       'CODE_NIVEAU', 'NIVEAU', 'DATE_INSCRIPTION', 'NIVEAU LMD',
       'NIVEAU LMD ET NON LMD', 'SYSTEME', 'DEPARTEMENT FORMATION'],
      dtype='object')

affichons les informations importantes du dataframe

In [ ]:
#fonction  les  informations du dataframe
def afficher_info(df):
    print("#==============Informations sur le DataFrame================ :")
    print(f"il y'a {df.shape[0]} lignes et {df.shape[1]} colonnes.")

afficher_info(data) 

#==============Informations sur le DataFrame================ :
il y'a 56115 lignes et 32 colonnes.


on remarque que l'année 2009-2010 a plus d'enregistrement que l'année 2001-2002 mais les deux jeux de données ont les meme colonnes 

cherchons les personnes inscrit avec le systeme LMD ces enregistrement seront considéres comme anormales 

In [6]:
#======= Anomalie1 : les personnes avec un systeme lmd inscrit avant 2010 =========
def anomalie1(df):
    anomalie1_df = df['SYSTEME'] == 'LMD'
    print(f"il y'a {anomalie1_df.sum()} personnes avec un systeme lmd inscrit .")
    
anomalie1(data)    

il y'a 36287 personnes avec un systeme lmd inscrit .


reperons les colonnes avec des valeurs manquantes 

In [7]:
#=========== Anomalie2 :  les valeurs manquantes dans les colonnes  ==========
def anomalie2(df):
    anomalie2_df = df.isnull().sum()
    anomalie2_df = anomalie2_df[anomalie2_df > 0]
    print(" le nombre de valeur manquante pour chaque colonne   :")
    print(anomalie2_df )
anomalie2(data) 

 le nombre de valeur manquante pour chaque colonne   :
NUMERO_TABLE           56115
INE                    39008
LIEU_DE_NAISSANCE         61
MAIL_INSTITUTIONNEL    33706
MENTION_BACC             338
ANNEE_BACC               291
SERIE_BACC              1186
REGION_DE_NAISSANCE     4042
COHORTE                50675
NIVEAU LMD             19828
dtype: int64


Petite remarque : la colonne numero_table n’a pas ete renseignee une seule fois valeur manquente(100%)

ici on vas reperer les année de naisance anormales 
tous les personnes ayant passant le bac  avant  14 ans ou apres 40 ans sont consideres comme anormales 
les personnes qui ont passes leur bac apres 30ans sont pas impossible mais tres rare 

In [9]:
#======anomalie3 :  les annéee de naissance pas bon ===============


def anomalie3(data):
    # Convertir la colonne 'DATE_DE_NAISSANCE' en datetime
    data['DATE_DE_NAISSANCE'] = pd.to_datetime(data['DATE_DE_NAISSANCE'], errors='coerce')
    # Extraire l'année de naissance
    data['annee_naissance'] = data['DATE_DE_NAISSANCE'].dt.year
    # Nettoyer et extraire uniquement les années de bac à 4 chiffres
    data['ANNEE_BACC'] = pd.to_numeric(
        data['ANNEE_BACC'].astype(str).str.extract(r'(\d{4})')[0], 
        errors='coerce'
    )
   # Calculer l'âge au bac uniquement pour les lignes valides
    data['age_bac'] = data['ANNEE_BACC'] - data['annee_naissance']
   # Rechercher les personnes ayant passé le bac avant 14 ans
    personnes_bac_avant_14_ans = data[data['age_bac'] < 14]
    personnes_bac_apres_40ans = data[data['age_bac'] > 40]
    personnes_bac_apres_30ans = data[data['age_bac'] > 30]
    print(f"Il y a {personnes_bac_avant_14_ans.shape[0]} personnes qui ont passé le bac avant 14 ans. voici leur année de naissance et l'année de bac : \t")
    print(personnes_bac_avant_14_ans[['DATE_DE_NAISSANCE', 'ANNEE_BACC']])
    print(f"Il y a {personnes_bac_apres_40ans.shape[0]} personnes qui ont passé le bac avant 40 ans. voici leur année de naissance et l'année de bac : \t")
    print(personnes_bac_apres_40ans[['DATE_DE_NAISSANCE', 'ANNEE_BACC']])
    print(f"Il y a {personnes_bac_apres_30ans.shape[0]} personnes qui ont passé le bac avant 30 ans.! pas impossible mais rare")

    
anomalie3(data)     

Il y a 25 personnes qui ont passé le bac avant 14 ans. voici leur année de naissance et l'année de bac : 	
      DATE_DE_NAISSANCE  ANNEE_BACC
2472         2007-12-31      2009.0
5834         1986-11-15      1990.0
6071         1984-03-08      1979.0
10257        1977-01-01      1987.0
16152        1981-12-31      1981.0
16305        1991-11-16      1991.0
16399        1980-08-18      1974.0
19871        1991-01-04      2004.0
21437        1992-03-21      1994.0
24569        1977-02-24      1977.0
25290        1977-06-27      1977.0
25899        1990-03-07      1990.0
26593        1987-08-30      1008.0
26789        1990-03-06      2000.0
28636        1983-11-20      1993.0
30509        1988-09-23      2000.0
40402        1978-12-31      1989.0
41618        1995-01-01      2007.0
42848        1968-09-24      1978.0
46880        1977-09-27      1990.0
47040        1974-09-28      1966.0
51438        1988-08-23      2000.0
51698        2010-10-24      2009.0
53553        1976-12-31      

Nous avons choisi de travailler avec l'age au moment de passer le bac  plutot qu'avec l'annee seule, car une  année peut etre anormale pour une personne et etre correcte pour une autre.l'erreur peut etre soit avec l'anne de naissance soit avec l'anne du bac.l'age permet de detcter les incoherences

dans la section suivante notre but est de reperer les doublons ou les personnes qui font deux formations ou plus

In [12]:
def anomalie4(data):
    # Compter les doublons exacts (lignes identiques)
    total = data.duplicated().sum()
    
    # Filtrer les étudiants apparaissant plusieurs fois avec le même numéro
    d = data[data.duplicated(subset=['NUMERO'], keep=False)]

    # Compter le nombre d'établissements différents pour chaque étudiant
    totale1 = d.groupby('NUMERO')['ETABLISSMENT_CODE'].nunique()

    # Compter combien d'étudiants ont étudié dans plusieurs établissements
    nb_personnes_2_formations = (totale1 == 2).sum()
    nb_personnes_3_formations = (totale1 == 3).sum()
    nb_personnes_4_formations = (totale1 == 4).sum()
    nb_personnes_plus_4 = (totale1 > 4).sum()

    print(f"Il y a {total} doublons exacts dans le DataFrame.")
    print(f"Il y a {nb_personnes_2_formations} personnes qui ont étudié dans 2 établissements différents la meme année.")
    print(f"Il y a {nb_personnes_3_formations} personnes qui ont étudié dans 3 établissements différents.")
    print(f"Il y a {nb_personnes_4_formations} personnes qui ont étudié dans 4 établissements différents.")
    print(f"Il y a {nb_personnes_plus_4} personnes qui ont étudié dans plus de 4 établissements différents.")
    
    
anomalie4(data)    


Il y a 0 doublons exacts dans le DataFrame.
Il y a 503 personnes qui ont étudié dans 2 établissements différents la meme année.
Il y a 2 personnes qui ont étudié dans 3 établissements différents.
Il y a 0 personnes qui ont étudié dans 4 établissements différents.
Il y a 0 personnes qui ont étudié dans plus de 4 établissements différents.


cherchons les mentions du bac incorrectes

In [13]:
#========== anomalie5 : les mentions au bac pas normales  ===========
def anomalie5(data):
    #les mentions au bac pas normales
    compteur = 0 
    mentions_normales  = ['AB', 'B', 'TB', 'PA']
    #compter le nombre de mentions anormales
    for i in data['MENTION_BACC']:
        if i not in mentions_normales:
            compteur+=1 
        
    print(f"il y'a {compteur} mentions anormales au bac")
    #afficher les mentions anormales
    mentions_anormales = data[~data['MENTION_BACC'].isin(mentions_normales)]['MENTION_BACC'].unique()
    print("les mentions anormales sont : ", mentions_anormales)
    
anomalie5(data)   

il y'a 1042 mentions anormales au bac
les mentions anormales sont :  [nan ' s' 'BI' '50' 'HO' 'da' '20' 'th' '-' 'mb' 'MB' 'ka' 'lo' 'po' 'do'
 'P' 'AS' 'TH' 'pi' 'p' 'no']


les personnes qui ont basse apres le bac apartir de 1998 auront comme serie de bac [L1B,S1A,S1,L1A,S3,S4,T1,STEG,SEA,F6,S5,S2,STIDD,LA,T2,L-AR,L2,L'1]
les personnes qui ont passe le bac avant 1998 auront comme seri series_bac_avant_1998 = ["A1", "A2", "A3", "A4", "C", "D", "E",  "F1", "F2", "F3", "F4" ,"F5", "F6", "F7",  "G1", "G2", "G3",  "T1", "T2",  "L'", "S'"  ]

In [14]:
#==========anomalie6 : les series de bac pas valides =======
def anomalie6(data):
    #les series de bac pas valides
    compteur = 0 
    compteur1 = 0
    series_normales_avant_98 = [ "A1", "A2", "A3", "A4",  "C", "D", "E","F1", "F2", "F3", "F4", "F5", "F6", "F7", "G1", "G2", "G3",  "T1", "T2",  "L'", "S'"  ]  
    series_normales_apres_98 = ["L1B", "S1A", "S1", "L1A", "S3", "S4", "T1", "STEG", "SEA", "F6","S5", "S2", "STIDD", "LA", "T2", "L-AR", "L2", "L'1"]         
     #convertir la colonne 'ANNEE_BACC' en entier
    data['ANNEE_BACC'] = pd.to_numeric(data['ANNEE_BACC'], errors='coerce')
    #compter le nombre de series anormales
    for _, i in data.iterrows():
        if i['SERIE_BACC'] not in series_normales_avant_98 and i['ANNEE_BACC'] < 1998:
            compteur+=1 
        if i['SERIE_BACC'] not in series_normales_apres_98 and i['ANNEE_BACC'] >= 1998:
            compteur1+=1    
        
    print(f"il y'a {compteur} qui ont passe leur bac avant  1998 qui une serie de bac pas normale")
    #afficher les series anormales
    series_anormales = data[~data['SERIE_BACC'].isin(series_normales_avant_98) & (data['ANNEE_BACC'] < 1998)]['SERIE_BACC'].unique()
    print("ces series sont : ", series_anormales)
    print(f"il y'a {compteur1} qui ont passe leur bac apres  1998 qui une serie de bac pas normale")
    #afficher les series anormales
    series_anormales1 = data[~data['SERIE_BACC'].isin(series_normales_apres_98) & (data['ANNEE_BACC'] >= 1998)]['SERIE_BACC'].unique()
    print("ces series sont : ", series_anormales1)
    
     
anomalie6(data)    

il y'a 568 qui ont passe leur bac avant  1998 qui une serie de bac pas normale
ces series sont :  [nan 'S2' 'A' 'L2' 'G' 'S5' 'B' "L'1" 'a3' 'S1' 'S' 'S3' 'ES' 'SB']
il y'a 5810 qui ont passe leur bac apres  1998 qui une serie de bac pas normale
ces series sont :  ['D' 'A3' 'G' 'C' nan 'S' 'A4' 'A' 'SNB' 'A2' 'G2' 'a3' 'B' "L'" 'E' 'SM'
 'F7' 'SB' 'BT' 'ES' 'SNA' 'F2']


affichage de tous les fonction creer etl'enregistre dans un fichier texte


In [15]:
#================= afficher toutes les anomalies ===================
def afficher_tous_les_anomalies(data):
        afficher_info(data)
        anomalie1(data)
        anomalie2(data)
        anomalie3(data)
        anomalie4(data)
        anomalie5(data)
        anomalie6(data)
        

afficher_tous_les_anomalies(data)

#==============Informations sur le DataFrame================ :
il y'a 56115 lignes et 34 colonnes.
il y'a 36287 personnes avec un systeme lmd inscrit .
 le nombre de valeur manquante pour chaque colonne   :
NUMERO_TABLE           56115
INE                    39008
LIEU_DE_NAISSANCE         61
MAIL_INSTITUTIONNEL    33706
MENTION_BACC             338
ANNEE_BACC               353
SERIE_BACC              1186
REGION_DE_NAISSANCE     4042
COHORTE                50675
NIVEAU LMD             19828
age_bac                  353
dtype: int64
Il y a 25 personnes qui ont passé le bac avant 14 ans. voici leur année de naissance et l'année de bac : 	
      DATE_DE_NAISSANCE  ANNEE_BACC
2472         2007-12-31      2009.0
5834         1986-11-15      1990.0
6071         1984-03-08      1979.0
10257        1977-01-01      1987.0
16152        1981-12-31      1981.0
16305        1991-11-16      1991.0
16399        1980-08-18      1974.0
19871        1991-01-04      2004.0
21437        1992-03-21      1